In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,13618.80,0.24,0.06,0.04,0.00,-0.00,0.00,-0.00
3,0.24,2680.43,-0.02,0.01,-0.00,0.00,-0.00,0.00
4,0.06,-0.02,426.74,0.00,0.00,-0.00,0.00,-0.00
5,0.04,0.01,0.00,60.75,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,8.15,0.00,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,1.08,-0.00,0.00
8,0.00,-0.00,0.00,0.00,0.00,-0.00,0.16,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.02


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00004,0.00000,-0.00000,0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00002,0.00003
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00001,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,-0.00000,0.00001,-0.00004,0.00001,0.00002,1.00000,-0.00003,0.00001
8,0.00000,-0.00002,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.3548058165317304

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.32032812e+07 8.79696105e+05 2.16814774e+04 4.21944614e+02
 7.12816580e+00 1.11135995e-01 1.92633808e-03 6.50183614e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999564,-0.029186,-0.004445,-0.000630,-0.000085,-0.000011,-0.000002,-2.067360e-07
1,0.029051,0.999197,-0.027346,-0.003774,-0.000507,-0.000067,-0.000010,-1.092307e-06
2,0.005215,0.027082,0.999188,-0.029134,-0.003816,-0.000507,-0.000076,-9.063440e-06
3,0.000887,0.004520,0.028868,0.998977,-0.034225,-0.004425,-0.000671,-7.844423e-05
4,0.000149,0.000757,0.004756,0.033860,0.998384,-0.044892,-0.006655,-7.733458e-04
5,0.000026,0.000132,0.000834,0.005844,0.044059,0.995975,-0.077341,-8.689295e-03
6,0.000006,0.000030,0.000188,0.001322,0.009857,0.075437,0.990265,-1.165546e-01
7,0.000001,0.000007,0.000044,0.000311,0.002317,0.017532,0.115535,9.931460e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004360836598644724,
 0.0008034550924327366,
 0.0008124468946822949,
 0.0010234945192492129,
 0.0016162752870444441,
 0.004024747885544189,
 0.009734717656468495,
 0.0068540245013640755]